<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2024TeamC/blob/main/Code/ETL2Datawarehouse_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kursarbeit Datenmanagement und -Archivierung WS 23/24**

Master-Code Name-CALICO:MA


**Erstellung der Datenbank**


*   Datenbankinitialisierung:



> Laden Libraries

In [1]:
import os
import pandas as pd
from functools import reduce
import sqlite3
from sqlite3 import Error
import csv
#import requests

>Google drive verbinden

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!ls drive/

Mounted at /content/drive
MyDrive  Shareddrives


>connect to repo

In [3]:
# Pfad zum Ordner
ordner_pfad = "/pfad/zum/ordner"
unterordner_name = "mein_unterordner"

# Vollständiger Pfad zum Unterordner
unterordner_pfad = os.path.join(ordner_pfad, unterordner_name)

# Überprüfen, ob der Unterordner existiert
if not os.path.exists(unterordner_pfad):
    # Wenn der Unterordner nicht existiert, erstellen Sie ihn
    os.makedirs(unterordner_pfad)
    print(f"Unterordner '{unterordner_name}' wurde erstellt.")
else:
    print(f"Unterordner '{unterordner_name}' existiert bereits.")

Unterordner 'mein_unterordner' wurde erstellt.


In [4]:
#definieren des Ordners in dem die repo gesichert werdern soll
google_drive_folder = "drive/MyDrive"
repo_colab_name = "calico_ma_repo"
#full_path_repo_colab = f"{google_drive_folder}+"//"+{repo_colab_name}"
full_path_repo_colab = os.path.join(google_drive_folder,repo_colab_name)
if not os.path.exists(full_path_repo_colab):
    os.makedirs(full_path_repo_colab)
    print(f"Ordner {full_path_repo_colab} ajngelegt")
else:
    print(f"Ordner {full_path_repo_colab} existiert bereits")

Ordner drive/MyDrive/calico_ma_repo existiert bereits


In [5]:
! apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [6]:
!git clone https://github.com/Fuenfgeld/DMA2024TeamC.git $full_path_repo_colab

fatal: destination path 'drive/MyDrive/calico_ma_repo' already exists and is not an empty directory.


In [7]:
!ls $full_path_repo_colab

Code  Data  Documentation  LICENSE  README.md


>Definition der Umgebnungsvariablen

>Verbindung zu CSV-Dateien herstellen und Kategorisierung nach der Tumorerkrankung
**AMENDMENT FOR ML: project_name = "CALICO_MA_ML"**

In [8]:
#repo_home = "/home/gunnar/DMA2024TeamC/" ## muss entsprechend angepasst werden für das lokale repo verzeichnis
repo_home = full_path_repo_colab+"/" ## repo in google drive
csv_data_folder = repo_home + "Data/Quelldaten/CSV_Data/"
project_name = "CALICO_MA_ML"
print(repo_home)
print (csv_data_folder)

drive/MyDrive/calico_ma_repo/
drive/MyDrive/calico_ma_repo/Data/Quelldaten/CSV_Data/




> Verbindung zu SQLite herstellen und Datebankinitialisierung Local




>The provided code is a Python script that establishes a connection to a SQLite database and creates a cursor object for executing SQL commands.
>The `create_connection_local` function is defined to establish a connection to a SQLite database. The function takes one parameter, `local_path`, which is the path to the database file. Inside the function, a `try` block is used to attempt to establish the connection. The `sqlite3.connect` function is called with the path to the database file, which is constructed by concatenating `local_path`, the string 'Data/CALICO_MA_Database/', and the global variable `project_name` with '.db' extension. If the connection is established successfully, the connection object is returned. However, the `print(sqlite3.version)` statement after the `return` statement will never be executed because the function exits as soon as it hits the `return` statement. If there is an error while establishing the connection, it is caught by the `except` block, and a message is printed to the console.
>After the `create_connection_local` function is defined, it is called with `repo_home` as the argument, and the returned connection object is stored in the `conn` variable.
>Next, a cursor object is created by calling the `cursor` method on the connection object. This cursor object, stored in the `cur` variable, can be used to execute SQL commands.
>Finally, a success message is printed to the console, indicating that the connection to the SQLite database was established successfully. The message includes the path to the database file.

In [9]:
def create_connection_local(local_path):
  conn = None;
  try:
    #Establishing the connection
    conn = sqlite3.connect(local_path+'Data/CALICO_MA_Database/'+project_name+'.db')
    return conn
    print(sqlite3.version)
  except Error as e:
    print("Error while connecting to sqlite", e)
conn = create_connection_local(repo_home)
# Creating a cursor object using the cursor() method
cur = conn.cursor()
print("Successfully Connected to SQLite Public Data Warehouse: "+repo_home+'Data/CALICO_MA_Database/'+project_name+'.db')


Successfully Connected to SQLite Public Data Warehouse: drive/MyDrive/calico_ma_repo/Data/CALICO_MA_Database/CALICO_MA_ML.db


> Quelldatenimport

This Python script is used to read multiple CSV files from different directories, create pandas DataFrames from them, and then concatenate these DataFrames based on their types.

Here's a step-by-step explanation:

1. Two lists are defined: `csvs` and `tumortypes`. `csvs` contains the names of different types of CSV files, and `tumortypes` contains the names of different types of tumors.

2. An empty dictionary `df` is created to store the DataFrames.

3. The script then enters a nested loop where it iterates over each tumor type and each CSV type. For each combination, it constructs a key (e.g., "procedures_Breast_Cancer") and a file path.

4. It reads the CSV file at the constructed path into a DataFrame, adds a new column `cancer_type` to the DataFrame, and stores the DataFrame in the `df` dictionary with the constructed key.

5. The DataFrame is also appended to a list in the `csv_dfs` dictionary corresponding to its CSV type.

6. After all CSV files have been read and their DataFrames stored, the script concatenates the DataFrames in each list in the `csv_dfs` dictionary. The result is a dictionary where each key is a CSV type and each value is a DataFrame that is the concatenation of all DataFrames of that CSV type.

7. Finally, the script prints the first few rows of each DataFrame in the `df` and `csv_dfs` dictionaries.

The commented-out code at the end of the script shows how individual CSV files were previously read into DataFrames. This code has been replaced by the loop that reads and concatenates all CSV files.

In [10]:
# Generate dataframes and add cancer_type information as a new column

csvs = ["procedures", "encounters", "immunizations", "medications", "observations", "patients"]

tumortypes = ["Breast_Cancer", "Lung_Cancer", "Colorectal_Cancer"]

df = {}
csv_dfs = {csv: [] for csv in csvs}

for tumor in tumortypes:
    for csv in csvs:
        key = f"{csv}_{tumor}"
        path = f"{csv_data_folder}{tumor}/{csv}.csv"
        df[key] = pd.read_csv(path, sep=",")
        df[key] = df[key].assign(cancer_type=tumor)

        # Add the dataframe to the list for this csv
        csv_dfs[csv].append(df[key])

# Check the sum of the number of rows in the original dataframes for each csv
for csv in csvs:
    total_rows = sum(len(df[csv + '_' + tumor]) for tumor in tumortypes)
    print(f"Total number of rows in original dataframes for {csv}: {total_rows}")
    print("\n")
# Concatenate the dataframes for each csv
for csv in csvs:
    csv_dfs[csv] = pd.concat(csv_dfs[csv])

# Check the number of rows and unique cancer types in each concatenated dataframe
for csv, df in csv_dfs.items():
    print(f"Concatenated DataFrame for {csv}:")
    print(f"Number of rows: {len(df)}")
    print(f"Unique cancer types: {df['cancer_type'].unique()}")
    print("\n")


Total number of rows in original dataframes for procedures: 2140


Total number of rows in original dataframes for encounters: 25836


Total number of rows in original dataframes for immunizations: 37878


Total number of rows in original dataframes for medications: 1258


Total number of rows in original dataframes for observations: 95849


Total number of rows in original dataframes for patients: 3084


Concatenated DataFrame for procedures:
Number of rows: 2140
Unique cancer types: ['Breast_Cancer' 'Lung_Cancer' 'Colorectal_Cancer']


Concatenated DataFrame for encounters:
Number of rows: 25836
Unique cancer types: ['Breast_Cancer' 'Lung_Cancer' 'Colorectal_Cancer']


Concatenated DataFrame for immunizations:
Number of rows: 37878
Unique cancer types: ['Breast_Cancer' 'Lung_Cancer' 'Colorectal_Cancer']


Concatenated DataFrame for medications:
Number of rows: 1258
Unique cancer types: ['Breast_Cancer' 'Lung_Cancer' 'Colorectal_Cancer']


Concatenated DataFrame for observations:
Numb

>Quality checks of the dataframes

In [11]:
# .nunique(axis=0) returns the number of unique values for each columns in the concanated dataframes.
for csv, df in csv_dfs.items():
    print(f"Quality for concatenated DataFrame {csv}:\n{df.nunique(axis=0)}")
    print("\n")



Quality for concatenated DataFrame procedures:
DATE                 1804
PATIENT               449
ENCOUNTER            1593
CODE                   37
DESCRIPTION            37
BASE_COST            2044
REASONCODE             11
REASONDESCRIPTION      11
cancer_type             3
dtype: int64


Quality for concatenated DataFrame encounters:
Id                     25836
START                  25521
STOP                   25720
PATIENT                 3084
ORGANIZATION            2268
PROVIDER                2806
PAYER                     10
ENCOUNTERCLASS             5
CODE                      15
DESCRIPTION               18
BASE_ENCOUNTER_COST        2
TOTAL_CLAIM_COST           2
PAYER_COVERAGE            15
REASONCODE                11
REASONDESCRIPTION         11
cancer_type                3
dtype: int64


Quality for concatenated DataFrame immunizations:
DATE           23245
PATIENT         3084
ENCOUNTER      23349
CODE              18
DESCRIPTION       18
BASE_COST          1
ca

In [12]:
#percentage of not null values in each column in the concanated dataframes.
for csv, df in csv_dfs.items():
    print(f"percentage of not null values in each column of concatenated DataFrame {csv}:\n{df.count()/len(df) * 100}")
    print("\n")


percentage of not null values in each column of concatenated DataFrame procedures:
DATE                 100.000000
PATIENT              100.000000
ENCOUNTER            100.000000
CODE                 100.000000
DESCRIPTION          100.000000
BASE_COST            100.000000
REASONCODE            74.439252
REASONDESCRIPTION     74.439252
cancer_type          100.000000
dtype: float64


percentage of not null values in each column of concatenated DataFrame encounters:
Id                     100.0000
START                  100.0000
STOP                   100.0000
PATIENT                100.0000
ORGANIZATION           100.0000
PROVIDER               100.0000
PAYER                  100.0000
ENCOUNTERCLASS         100.0000
CODE                   100.0000
DESCRIPTION            100.0000
BASE_ENCOUNTER_COST    100.0000
TOTAL_CLAIM_COST       100.0000
PAYER_COVERAGE         100.0000
REASONCODE               2.2488
REASONDESCRIPTION        2.2488
cancer_type            100.0000
dtype: float64




In [13]:
#Delete Tables in case they exist
cur.execute("DROP TABLE IF EXISTS procedures")
cur.execute("DROP TABLE IF EXISTS encounters")
cur.execute("DROP TABLE IF EXISTS immunizations")
cur.execute("DROP TABLE IF EXISTS medications")
cur.execute("DROP TABLE IF EXISTS observations")
cur.execute("DROP TABLE IF EXISTS patients")
cur.execute("DROP TABLE IF EXISTS facts_table")

>Create tables including facts_table
**AMENDMENTS FOR ML: COUNTY as STRING, ZIP as STRING**

In [14]:
# Create tables including facts_table

sql_table = {}

create_table_command = """CREATE TABLE IF NOT EXISTS patients (
                           Id         STRING PRIMARY KEY,
                           BIRTHDATE  DATE,
                           DEATHDATE  DATE,
                           SSN        STRING,
                           DRIVERS    STRING,
                           PASSPORT   STRING,
                           PREFIX     STRING,
                           FIRST      STRING,
                           LAST       STRING,
                           SUFFIX     STRING,
                           MAIDEN     STRING,
                           MARITAL    STRING,
                           RACE       STRING,
                           ETHNICITY  STRING,
                           GENDER     STRING,
                           BIRTHPLACE STRING,
                           ADDRESS    STRING,
                           CITY       STRING,
                           STATE      STRING,
                           COUNTY     STRING,
                           ZIP        STRING,
                           LAT        DECIMAL(6,2),
                           LON        DECIMAL(6,2),
                           HEALTHCARE_EXPENSES DECIMAL(9,2),
                           HEALTHCARE_COVERAGE DECIMAL(9,2),
                           cancer_type STRING
  );

CREATE TABLE IF NOT EXISTS encounters (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ORGANIZATION STRING,
                           PROVIDER STRING,
                           PAYER STRING,
                           ENCOUNTERCLASS STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_ENCOUNTER_COST DECIMAL(9,2),
                           TOTAL_CLAIM_COST DECIMAL(9,2),
                           PAYER_COVERAGE DECIMAL(9,2),
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
);



CREATE TABLE IF NOT EXISTS observations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT) REFERENCES patients (Id),
                           FOREIGN KEY (Encounter) REFERENCES encounters (Id)
);


CREATE TABLE IF NOT EXISTS medications (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           PAYER STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST DECIMAL(9,2),
                           PAYER_COVERAGE DECIMAL(9,2),
                           DISPENSES DECIMAL(9,2),
                           TOTALCOST DECIMAL(9,2),
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
  );

CREATE TABLE IF NOT EXISTS procedures (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST DECIMAL(9,2),
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT) REFERENCES patients (Id),
                           FOREIGN KEY (Encounter) REFERENCES encounters (Id)
                       );

CREATE TABLE IF NOT EXISTS immunizations (
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST DECIMAL(9,2),
                           cancer_type STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id)
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id)
  );

CREATE TABLE IF NOT EXISTS facts_table (
                          cancer_type STRING,
                          patient_ID STRING,
                          encounter_ID STRING,
                          patient_LAT        DECIMAL(6,2),
                          patient_LON        DECIMAL(6,2),
                          patient_COUNTY     STRING,
                          patient_ZIP        STRING,
                          patient_HEALTHCARE_EXPENSES DECIMAL(9,2),
                          patient_HEALTHCARE_COVERAGE DECIMAL(9,2),
                          patient_BIRTHDATE DATE,
                          observations_CODE STRING,
                          observations_VALUE STRING,
                          observations_DATE  DATE,
                          observations_AGE DECIMAL(6,2),
                          procedures_BASE_COST DECIMAL(11,2),
                          medications_TOTALCOST DECIMAL(11,2),
                          immunizations_BASE_COST DECIMAL(11,2),
                          encounter_BASE_ENCOUNTER_COST DECIMAL(11,2),
                          FOREIGN KEY (patient_ID) REFERENCES patients (Id),
                          FOREIGN KEY (patient_ID) REFERENCES observations (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES observations (ENCOUNTER),
                          FOREIGN KEY (patient_ID) REFERENCES encounters (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES encounters (Id),
                          FOREIGN KEY (patient_ID) REFERENCES procedures (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES procedures (ENCOUNTER),
                          FOREIGN KEY (patient_ID) REFERENCES medications (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES medications (ENCOUNTER),
                          FOREIGN KEY (patient_ID) REFERENCES immunizations (PATIENT),
                          FOREIGN KEY (encounter_ID) REFERENCES immunizations (ENCOUNTER)

);
"""
# Execute the SQL command
cur.executescript(create_table_command)

conn.commit()

In [15]:
#Insert data from the concanated data frames into the tables

for csv, df in csv_dfs.items():
    df.to_sql(csv, conn, if_exists='append', index=False)

conn.commit()

In [16]:
#Check if data is in the tables. 10 random rows are selected from each table

for csv in csvs:
    cur.execute(f"SELECT * FROM {csv} ORDER BY RANDOM() LIMIT 10")
    rows = cur.fetchall()
    df_random = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
    print(f"SQL tables from {repo_home}Data/CALICO_MA_Database/{project_name}.db' \n\n{csv} {df_random} ")


SQL tables from drive/MyDrive/calico_ma_repo/Data/CALICO_MA_Database/CALICO_MA_ML.db' 

procedures                    DATE                               PATIENT  \
0  2014-12-21T23:26:40Z  bbd3db10-bbdb-8d43-61aa-ad2dab78459a   
1  2014-05-12T09:00:42Z  d1b42847-0bae-e1be-22f5-2291d5b896f4   
2  2012-04-10T21:56:16Z  d42d555a-8825-38af-e8c3-1b7f503730e9   
3  2014-12-11T08:52:20Z  b132451f-94cd-9d40-6103-34741a65d610   
4  2015-12-13T10:41:44Z  61cce672-d715-d945-c67e-d1b357904d1e   
5  2015-05-03T22:31:55Z  530d950d-71f5-05d9-de46-0742db31191c   
6  2016-02-26T20:33:47Z  692885aa-deee-6222-f202-743814a82ee5   
7  2018-09-16T14:47:26Z  bcd6bed4-0458-5270-2a64-d9383e7478b3   
8  2012-12-13T14:13:42Z  8417ea73-2556-9bc3-817e-964762bda4ae   
9  2020-07-16T14:10:54Z  038eb669-b744-c2bc-18c6-6e37068c722f   

                              ENCOUNTER       CODE  \
0  769fd6e3-46cd-9ebe-7895-eb6f521aa82e   71651007   
1  903ded85-c1e8-a8bf-0a4c-d2dc0bea0c2f  180325003   
2  3f7c694d-b3e4-2b0b-b

### Die Daten werden aus den verschiedenen Tabellen extrahiert und ggf. transformiert
 >Die Daten aus **Patients** werden erstmal in die Faktentabelle übertragen. Es handelt sich um die Krebsart, die Patienten ID, die geographische Koordinaten, die Healthcarekosten, die vom Patient getragen werden und diejenigen, die von der Krankenversicherung bezahlt werden, und zum Ende das Geburtsdatum.
Damit die Faktentabelle keine Redundanzen aufweist, werden die Encounter ID nicht mehr einzeln gespeichert. Stattdessen werden in der Faktentabelle für jeden Patienten die gesamten Kosten gespeichert, die in jeder Kategorie (Encounters, Observations, Immunizations, Procedures umd Medications) verursacht wurden.  
Die Transaktion in der Faktentabelle der Datenbank wird mit commit beendet.
**AMENDMENT FOR ML: patient_COUNTY and patient_ZIP included**

ACHTUNG: 53% der Patienten haben keine ZIP



In [17]:
# TEST how many patients undergo Procedures
#cur.execute('''SELECT COUNT(*) from procedures GROUP BY procedures.PATIENT''')
#records = cur.fetchall()
#i=1
#for row in records:
#  print(f'patient', i, 'undergoes ', row,' procedures')
#  i=i+1

In [18]:
# Datenextraktion aus der Patientstabelle
cur.execute('''INSERT INTO facts_table
                    ( cancer_type, patient_ID, encounter_ID, patient_LAT, patient_LON, patient_COUNTY, patient_ZIP, patient_HEALTHCARE_EXPENSES , patient_HEALTHCARE_COVERAGE, patient_BIRTHDATE)
                    SELECT
                        p.cancer_type AS cancer_type,
                        p.Id AS patient_ID,
                        NULL AS encounter_ID,  -- Replace with the actual encounter_ID or set to NULL if not applicable
                        p.LAT AS patient_LAT,
                        p.LON AS patient_LON,
                        p.COUNTY AS patient_COUNTY,
                        p.ZIP AS patient_ZIP,
                        p.HEALTHCARE_EXPENSES AS patient_HEALTHCARE_EXPENSES,
                        p.HEALTHCARE_COVERAGE AS patient_HEALTHCARE_COVERAGE,
                        p.BIRTHDATE AS patient_BIRTHDATE
                    FROM patients p
                    ;''')

# Ende der Transaktion
conn.commit()

# Schnelle Überprüfung der Datenübertragung ins die Faktentabelle
cur.execute('''SELECT patient_ID, patient_COUNTY, patient_ZIP, patient_LON, patient_LAT, patient_HEALTHCARE_EXPENSES, patient_HEALTHCARE_COVERAGE, patient_BIRTHDATE FROM facts_table''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

('d2061cc7-bee0-0e6c-3ac4-15c197c474e0', 'Middlesex County', 2140, -71.11326024935221, 42.35992586937376, 1475230, 4244.639999999999, '1956-06-22')
('073d8e80-ff90-1c8d-57e4-29bfca52c87f', 'Worcester County', 1535, -72.02631680460482, 42.25784546955933, 1489125, 4016.36, '1964-08-28')
('e1ff7e68-4097-9faf-514d-e4cfcfdf252e', 'Worcester County', 1545, -71.8184488503781, 42.204289674137854, 57142.44, 2103.2, '1998-08-28')
('a0f679cc-875f-dd72-ed13-9ca863ec6cf3', 'Norfolk County', None, -70.90408406035422, 42.15026973562508, 408520, 2709, '2004-01-20')
('e4166a9c-f7c5-bef7-ea43-b96281a2d586', 'Worcester County', 1609, -71.78487317237207, 42.2920819230455, 320400, 2970.68, '2006-03-23')
('ee537126-7509-05de-2349-ab19604545d8', 'Suffolk County', 2134, -71.04135024287056, 42.347570821295285, 784465, 2738.96, '1991-05-21')
('826d5ce3-2b23-8bd8-b352-a77fdd083d68', 'Middlesex County', 1890, -71.14747985490884, 42.52234296982323, 1425900, 3841.36, '1964-07-20')
('0edfef0f-fa74-ab35-1122-4c537bb9

Die Daten werden aus **Observations** eintragen:
nur die Tuples mit dem neuesten Datum werden berücksichtigt. Gespeichert werden die Art der Observation, in unserem Fall **nur den QALY**, den neuesten Wert und das entsprechende Datum. Ausserdem wird das Alter am Tag der letzten Observation berechnet und eingetragen. Die Transaktion wird mit commit beendet. Das Speichern der neuen Einträge wird überprüft.

In [19]:
# Datenextraktion aus der Observationstabelle
cur.execute('''UPDATE
                  facts_table
               SET
                  observations_CODE = obs.CODE,
                  observations_VALUE = obs.maxVALUE,
                  observations_DATE = obs.maxDATE,
                  observations_AGE = obs.maxDATE- patient_BIRTHDATE
               FROM
                    (SELECT PATIENT, CODE, MAX(VALUE) as maxVALUE, MAX(observations.DATE) as maxDATE
                    FROM observations
                    WHERE CODE LIKE "QALY"
                    GROUP BY PATIENT
                    ) AS obs
                WHERE
                  facts_table.patient_ID = obs.PATIENT
                    ;''')

# Ende der Transaktion
conn.commit()

# Schnelle Überprüfung der Datenübertragung ins die Faktentabelle
cur.execute('''SELECT patient_ID, observations_CODE, observations_VALUE, patient_BIRTHDATE, observations_DATE, observations_AGE  FROM facts_table ''')
records = cur.fetchall()
# show 10 rows
for row in records[:30]:
  print(row)

print("\n")

#cur.execute(''' SELECT COUNT(DISTINCT PATIENT) FROM observations''')
#records = cur.fetchall()
#print(f"#patients in observations: {records[-1][-1]}")

#cur.execute(''' SELECT PATIENT, MAX(VALUE), MAX(DATE) FROM observations GROUP BY PATIENT ''')
#records = cur.fetchall()
#for row in records[:50]:
#  print(row)
#print(records[-1][-1])

#print("\n")
#cur.execute(''' SELECT PATIENT, CODE, VALUE, DATE FROM observations WHERE CODE LIKE "DALY" AND VALUE >= 1 ''')
#records = cur.fetchall()
#for row in records[:60]:
#  print(row)
#print("\n")

#cur.execute(''' SELECT PATIENT, CODE, MAX(VALUE), MAX(DATE) FROM observations WHERE PATIENT = "950a56cd-d2d5-785c-d80c-c126912fbf88" AND CODE LIKE "QALY"''')
#records = cur.fetchall()
#for row in records[:60]:
#  print(row)
#print("\n")


#print("\n")


('d2061cc7-bee0-0e6c-3ac4-15c197c474e0', 'QALY', 63, '1956-06-22', '2020-06-22T10:47:46Z', 64)
('073d8e80-ff90-1c8d-57e4-29bfca52c87f', 'QALY', 55, '1964-08-28', '2020-08-28T16:45:45Z', 56)
('e1ff7e68-4097-9faf-514d-e4cfcfdf252e', 'QALY', 21, '1998-08-28', '2020-08-28T07:43:32Z', 22)
('a0f679cc-875f-dd72-ed13-9ca863ec6cf3', 'QALY', 15, '2004-01-20', '2020-01-20T15:27:37Z', 16)
('e4166a9c-f7c5-bef7-ea43-b96281a2d586', 'QALY', 13, '2006-03-23', '2020-03-23T22:58:03Z', 14)
('ee537126-7509-05de-2349-ab19604545d8', 'QALY', 28, '1991-05-21', '2020-05-21T01:23:21Z', 29)
('826d5ce3-2b23-8bd8-b352-a77fdd083d68', 'QALY', 55, '1964-07-20', '2020-07-20T05:08:46Z', 56)
('0edfef0f-fa74-ab35-1122-4c537bb9765d', 'QALY', 22, '1997-05-05', '2020-05-04T22:31:18Z', 23)
('0fe7e161-0f29-2b70-8e3d-757b15026295', 'QALY', 10, '2008-12-26', '2019-12-26T13:49:03Z', 11)
('478e120c-67a4-4375-646f-c41e74f72478', 'QALY', 49, '1971-01-02', '2020-01-02T11:38:08Z', 49)
('622b5d81-6868-03d0-85b2-1d5bb908f6a5', 'QALY', 1

Die Daten werden aus **Encounters** eintragen. Es handelt sich um die Summe aller Encounter Kosten für jeden einzelnen Patienten.
Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen.
Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft.


In [20]:
# Datenextraktion aus der Encounterstabelle und Berechnung der Summe aller Encounterkosten für jeden Patienten
cur.execute('''UPDATE
                  facts_table
               SET
                  encounter_Base_Encounter_Cost = enc.TOTALCOST
               FROM
                    (SELECT PATIENT, SUM(BASE_ENCOUNTER_COST) as TOTALCOST
                    FROM encounters, facts_table
                    WHERE facts_table.patient_ID = encounters.PATIENT
                    GROUP BY PATIENT
                    ) AS enc
                WHERE
                  facts_table.patient_ID = enc.PATIENT
                    ;''')

# Ende der Transaktion
conn.commit()

# Falls es keine Encounterkosten gibt, wird der Wert 0 stattdessen eingetragen
cur.execute('''UPDATE
                  facts_table
               SET
                  encounter_Base_Encounter_Cost = 0
                WHERE encounter_Base_Encounter_Cost IS NULL
                    ;''')

# Ende der Transaktion
conn.commit()

# Kurze Überprüfung
cur.execute('''SELECT patient_ID, encounter_BASE_ENCOUNTER_COST FROM facts_table''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)



('d2061cc7-bee0-0e6c-3ac4-15c197c474e0', 1291.6000000000001)
('073d8e80-ff90-1c8d-57e4-29bfca52c87f', 1033.28)
('e1ff7e68-4097-9faf-514d-e4cfcfdf252e', 1033.28)
('a0f679cc-875f-dd72-ed13-9ca863ec6cf3', 1291.6000000000001)
('e4166a9c-f7c5-bef7-ea43-b96281a2d586', 1291.6000000000001)
('ee537126-7509-05de-2349-ab19604545d8', 387.48)
('826d5ce3-2b23-8bd8-b352-a77fdd083d68', 1033.28)
('0edfef0f-fa74-ab35-1122-4c537bb9765d', 904.1199999999999)
('0fe7e161-0f29-2b70-8e3d-757b15026295', 1420.7600000000002)
('478e120c-67a4-4375-646f-c41e74f72478', 645.8)


In [21]:
cur.execute('''SELECT patient_ID, encounter_BASE_ENCOUNTER_COST, SUM(BASE_ENCOUNTER_COST)
                FROM facts_table, encounters WHERE facts_table.patient_ID = encounters.PATIENT
                GROUP BY patient_ID''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

('001c3804-81e8-740e-dd91-973b584a2dd9', 387.48, 387.48)
('002a707d-ba35-3d1a-46c0-ad8a98a22137', 774.9599999999999, 774.9599999999999)
('003c0d44-9dfc-ca91-6121-373a8e265bd4', 1446.5800000000002, 1446.5800000000002)
('008b26ab-5c12-7a5b-cda4-f54484d5f749', 1291.6000000000001, 1291.6000000000001)
('009121bf-a672-8942-443e-85e18a33f766', 1162.44, 1162.44)
('00e798f6-7d02-8485-c5fd-4b6bfb114c69', 516.64, 516.64)
('00edcb46-afd4-4687-c16a-f4e20dfc3e92', 1188.26, 1188.26)
('0103a559-910a-03df-6117-ec429eeb4ac9', 774.9599999999999, 774.9599999999999)
('01518fe2-bd76-d1e9-a656-1e7eae35dd17', 1446.5800000000002, 1446.5800000000002)
('01581211-2e47-027b-71a0-dcbf0f4e6622', 1317.42, 1317.42)


Die Daten werden aus **Medications** eintragen. Es handelt sich um die Summe aller MedikationsKosten für jeden einzelnen Patienten.
Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen.
Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft.

In [22]:
cur.execute('''     UPDATE facts_table SET medications_TotalCost = med.TOTALCOST
                    FROM
                    (SELECT PATIENT, SUM(Totalcost) as TOTALCOST
                    FROM medications, facts_table
                    WHERE facts_table.patient_ID = medications.PATIENT
                    GROUP BY PATIENT) as med
                    WHERE facts_table.patient_ID = med.PATIENT
                    ;''')
conn.commit()

In [23]:
cur.execute('''UPDATE
                  facts_table
               SET
                  medications_TOTALCOST = 0
                WHERE medications_TOTALCOST IS NULL
                    ;''')
conn.commit()

In [24]:
cur.execute('''SELECT patient_ID, medications_TOTALCOST, SUM(TOTALCOST)
                FROM facts_table, medications WHERE facts_table.patient_ID = medications.PATIENT
                GROUP BY patient_ID''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

('003c0d44-9dfc-ca91-6121-373a8e265bd4', 30329.850000000002, 30329.85)
('01d6dc15-6313-5327-8491-49be72f40d59', 113865.26, 113865.26000000001)
('023c4bb2-6dc3-972e-379a-af19b8a2392d', 117956.49000000002, 117956.49)
('02db7f22-8617-0cf7-fa10-d820d596a81a', 101166.45000000001, 101166.45000000001)
('030287cd-dbd1-d7e1-6959-cfdfa0e271bd', 230.28, 230.28)
('037404e1-0c87-534d-0fe2-e21ef20640f6', 70052.88, 70052.88)
('038eb669-b744-c2bc-18c6-6e37068c722f', 49598.27, 49598.270000000004)
('05023597-744e-9369-c390-e4090447a247', 63036.16, 63036.16)
('05a3a33e-e9b1-0aaa-66f4-f0e2b57dce04', 3219.0699999999997, 3219.0699999999997)
('072229fb-e8b1-a9ff-61b1-0cdab70c3c4c', 6615.34, 6615.34)


Die Daten werden aus **Procedures** eintragen. Es handelt sich um die Summe aller Prozedurenkosten für jeden einzelnen Patienten. Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen. Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft

In [25]:
cur.execute('''     UPDATE facts_table SET procedures_BASE_COST = proc.TOTALCOST
                    FROM
                    (SELECT PATIENT, SUM(BASE_COST) as TOTALCOST
                    FROM procedures, facts_table
                    WHERE facts_table.patient_ID = procedures.PATIENT
                    GROUP BY PATIENT) as proc
                    WHERE facts_table.patient_ID = proc.PATIENT
                    ;''')
conn.commit()

In [26]:
cur.execute('''UPDATE
                  facts_table
               SET
                  procedures_BASE_COST = 0
                WHERE procedures_BASE_COST IS NULL
                    ;''')
conn.commit()

In [27]:
cur.execute('''SELECT patient_ID, procedures_Base_COST, SUM(Base_COST)
                FROM facts_table, procedures WHERE facts_table.patient_ID = procedures.PATIENT
                GROUP BY patient_ID''')
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

('003c0d44-9dfc-ca91-6121-373a8e265bd4', 297586.43, 297586.43)
('00edcb46-afd4-4687-c16a-f4e20dfc3e92', 23882.18, 23882.18)
('01518fe2-bd76-d1e9-a656-1e7eae35dd17', 22514.35, 22514.35)
('01581211-2e47-027b-71a0-dcbf0f4e6622', 34044.95, 34044.95)
('01d6dc15-6313-5327-8491-49be72f40d59', 405522.31, 405522.31)
('023c4bb2-6dc3-972e-379a-af19b8a2392d', 56170.43, 56170.43)
('030287cd-dbd1-d7e1-6959-cfdfa0e271bd', 34779.04, 34779.04)
('037404e1-0c87-534d-0fe2-e21ef20640f6', 28800.590000000004, 28800.59)
('038eb669-b744-c2bc-18c6-6e37068c722f', 225341.52000000002, 225341.52)
('05398088-a847-099e-3c61-c6153d83e024', 35182.2, 35182.2)


Die Daten werden aus **Immunizations** eintragen. Es handelt sich um die Summe aller Immunizationskosten für jeden einzelnen Patienten. Falls dieser Wert nicht existiert, wird an ihrer Stelle 0 eingetragen. Jede Transaktion wird mit commt beendet. Das Speichern der neuen Einträge in die Faktentabelle wird überprüft

In [28]:
cur.execute('''     UPDATE facts_table SET immunizations_Base_Cost = im.TOTALCOST
                    FROM
                    (SELECT PATIENT, SUM(Base_Cost) as TOTALCOST
                    FROM immunizations, facts_table
                    WHERE facts_table.patient_ID = immunizations.PATIENT
                    GROUP BY PATIENT) as im
                    WHERE facts_table.patient_ID = im.PATIENT
                    ;''')
conn.commit()

In [29]:
cur.execute('''UPDATE
                  facts_table
               SET
                  immunizations_BASE_COST = 0
                WHERE immunizations_BASE_COST IS NULL
                    ;''')
conn.commit()

In [30]:
cur.execute(''' SELECT patient_ID, immunizations_BASE_COST, SUM(BASE_COST) FROM facts_table, immunizations
                WHERE facts_table.patient_ID = immunizations.PATIENT
                GROUP BY patient_ID''' )
records = cur.fetchall()
# show 10 rows
for row in records[:10]:
  print(row)

('001c3804-81e8-740e-dd91-973b584a2dd9', 562.08, 562.08)
('002a707d-ba35-3d1a-46c0-ad8a98a22137', 1264.68, 1264.68)
('003c0d44-9dfc-ca91-6121-373a8e265bd4', 1405.2, 1405.2)
('008b26ab-5c12-7a5b-cda4-f54484d5f749', 1545.72, 1545.72)
('009121bf-a672-8942-443e-85e18a33f766', 1545.72, 1545.72)
('00e798f6-7d02-8485-c5fd-4b6bfb114c69', 983.64, 983.64)
('00edcb46-afd4-4687-c16a-f4e20dfc3e92', 1545.72, 1545.72)
('0103a559-910a-03df-6117-ec429eeb4ac9', 1264.68, 1264.68)
('01518fe2-bd76-d1e9-a656-1e7eae35dd17', 1826.76, 1826.76)
('01581211-2e47-027b-71a0-dcbf0f4e6622', 1686.24, 1686.24)


> Werte in der Faktentabelle haben Komma statt Dot für die Dezimale. Konvertierung dieser.


In [31]:
cur.execute(''' UPDATE facts_table SET
patient_LAT = REPLACE(patient_LAT,',','.'),
patient_LON = REPLACE(patient_LON,',','.'),
patient_HEALTHCARE_EXPENSES = REPLACE(patient_HEALTHCARE_EXPENSES,',','.'),
patient_HEALTHCARE_COVERAGE = REPLACE(patient_HEALTHCARE_COVERAGE,',','.')
;''')
conn.commit()

##Das Datawarehouse ist fertig ausgefüllt.
Hier werden die ersten Tupeln der Faktentabelle angezeigt.
Der "None"-Eintrag entspricht der Spalte für Encounters. Diese Spalte wird nicht benutzt, deshalb wurde sie nicht ausgefüllt.

In [32]:
cur.execute('''SELECT * FROM facts_table
 ORDER BY RANDOM()''')
records = cur.fetchall()
# show 4 rows
for row in records[:30]:
  print(row)

('Colorectal_Cancer', '2e370d46-d229-7c08-4873-d5a9bcd699d9', None, 42.033064619444, -70.948397832431, 'Plymouth County', None, 1218210, 3033.89, '1970-05-10', 'QALY', 49, '2020-05-10T19:26:58Z', 50, 17229.31, 0, 1124.16, 723.29)
('Breast_Cancer', 'b05f9dd9-3e1e-88ad-5f94-ead265930ccf', None, 42.7101734274005, -71.0827502980551, 'Essex County', 1830, 1773540, 6080.36, '1938-07-14', 'QALY', 81, '2020-07-14T20:13:30Z', 82, 0, 0, 1405.2, 1162.44)
('Colorectal_Cancer', '5273f717-3833-0d7a-c9a5-e65a3d12e727', None, 41.6509934009745, -70.2707877404798, 'Barnstable County', None, 159350, 3402.21, '1968-07-30', 'QALY', 51, '2020-07-30T05:49:09Z', 52, 13181.17, 0, 1264.68, 852.4499999999999)
('Lung_Cancer', 'c8a12262-bd17-8ce0-50ae-cd30d2b0115a', None, 42.281270998356, -71.0455594186376, 'Norfolk County', 2186, 801755, 2378.12, '1987-09-19', 'QALY', 32, '2020-09-19T09:26:02Z', 33, 0, 0, 562.08, 387.48)
('Breast_Cancer', '8beb4bf0-b9ab-3af7-af5a-7a26a8734b26', None, 42.5504441928801, -71.8070581

In [33]:
print(pd.read_sql_query("PRAGMA table_info('facts_table')", conn))

    cid                           name           type  notnull dflt_value  pk
0     0                    cancer_type         STRING        0       None   0
1     1                     patient_ID         STRING        0       None   0
2     2                   encounter_ID         STRING        0       None   0
3     3                    patient_LAT   DECIMAL(6,2)        0       None   0
4     4                    patient_LON   DECIMAL(6,2)        0       None   0
5     5                 patient_COUNTY         STRING        0       None   0
6     6                    patient_ZIP         STRING        0       None   0
7     7    patient_HEALTHCARE_EXPENSES   DECIMAL(9,2)        0       None   0
8     8    patient_HEALTHCARE_COVERAGE   DECIMAL(9,2)        0       None   0
9     9              patient_BIRTHDATE           DATE        0       None   0
10   10              observations_CODE         STRING        0       None   0
11   11             observations_VALUE         STRING        0  

# Bereinigung der Faktentabelle

## Delele all rows with Age = 0

In [34]:
cur.execute('''SELECT COUNT(*)  FROM facts_table
 WHERE observations_AGE == 0''')
result = cur.fetchall()
print(f"Number of rows where observations_AGE == 0 : {result[-1][-1]}")
print("\n")

cur.execute(''' DELETE FROM facts_table WHERE observations_AGE = 0
;''')
conn.commit()

cur.execute('''SELECT COUNT(*)  FROM facts_table
 WHERE observations_AGE == 0''')
result = cur.fetchall()
print(f"Nach Bereinigung: Number of rows where observations_AGE == 0 : {result[-1][-1]}")
print("\n")


Number of rows where observations_AGE == 0 : 30


Nach Bereinigung: Number of rows where observations_AGE == 0 : 0




##Close Database connection

In [35]:
conn.close()
print("The Sqlite connection is closed")

The Sqlite connection is closed
